In [23]:
# pip install torch transformers datasets  scikit-learn
# pip install transformers[torch]

In [24]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder

# 🔧 Step 1: 数据集准备

In [25]:
""" texts = ["我今天很开心", "心情有点低落", "天气不错，但工作好累"]
labels = ["快乐", "悲伤", "中立"]

# 标签编码
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# 转化为Hugging Face格式
dataset = Dataset.from_dict({"text": texts, "label": encoded_labels}) """


' texts = ["我今天很开心", "心情有点低落", "天气不错，但工作好累"]\nlabels = ["快乐", "悲伤", "中立"]\n\n# 标签编码\nlabel_encoder = LabelEncoder()\nencoded_labels = label_encoder.fit_transform(labels)\n\n# 转化为Hugging Face格式\ndataset = Dataset.from_dict({"text": texts, "label": encoded_labels}) '

In [ ]:
#ChnSentiCorp 常用的中文情感分类数据集(电商产品向)
from datasets import load_dataset
dataset = load_dataset("clue", "tnews")
print(dataset['train'][0])

In [ ]:
#XNLI (Cross-lingual NLI Corpus)数据集
dataset = load_dataset("xnli", "zh")
print(dataset['train'][0])

Generating validation split: 100%|██████████| 2490/2490 [00:00<00:00, 698489.63 examples/s]

{'premise': '从 概念 上 看 , 奶油 收入 有 两 个 基本 方面 产品 和 地理 .', 'hypothesis': '产品 和 地理 是 什么 使 奶油 抹 霜 工作 .', 'label': 1}


In [29]:
print(dataset['train'][:5])


{'premise': ['从 概念 上 看 , 奶油 收入 有 两 个 基本 方面 产品 和 地理 .', '你 知道 在 这个 季节 , 我 猜 在 你 的 水平 你 把 他们 丢到 下 一个 水平 , 如果 他们 决定 召回 的 家长 队 , 勇士 队 决定 打电话 召回 一个 家伙 从 三 个 a , 然后 一个 双人 上 去. 取代 他 和 一个 男人 去 取代 他', '我们 的 一个 号码 会 非常 详细 地 执行 你 的 指示', '你 怎么 知道 的 ? 所有 这些 都 是 他们 的 信息 .', '是 啊 , 我 告诉 你 , 如果 你 去 买 一些 网球鞋 , 我 可以 看到 为什么 现在 你 知道 他们 是 起床 在 百 美元 范围 .'], 'hypothesis': ['产品 和 地理 是 什么 使 奶油 抹 霜 工作 .', '如果 人们 记得 的 话 , 你 就 会 把 事情 弄 丢 了 .', '我 团队 的 一个 成员 将 非常 精确 地 执行 你 的 命令', '这些 信息 属于 他们 .', '网球鞋 有 一 系列 的 价格 .'], 'label': [1, 0, 0, 0, 1]}



# 🔧 Step 2: 加载预训练模型

In [27]:

model_name = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



# 🔧 Step 3: 数据预处理

In [30]:

def preprocess_data(example):
    encoding = tokenizer(example['premise'], padding='max_length', truncation=True, max_length=128)
    encoding['label'] = example['label']
    return encoding

dataset = dataset.map(preprocess_data)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map: 100%|██████████| 2490/2490 [00:00<00:00, 3013.23 examples/s]


# 🔧 Step 4: 模型训练

In [ ]:

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset
)

trainer.train()


Epoch,Training Loss,Validation Loss



# 🔧 Step 5: 模型推理

In [ ]:

def predict(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    return label_encoder.inverse_transform([predicted_label])[0]

# 测试推理
display(predict("今天真是开心的一天！"))
display(predict("我有点难过"))
display(predict("心情很差"))
display(predict("天气很好，但我很累"))

np.str_('悲伤')

np.str_('悲伤')

np.str_('悲伤')

np.str_('悲伤')